In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install biopython

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.4 MB/s eta 0:00:00


In [2]:
pip install git+https://github.com/HeliXonProtein/OmegaFold.git

  Cloning https://github.com/HeliXonProtein/OmegaFold.git to /tmp/pip-req-build-51fbp8cw
  Running command git clone --filter=blob:none --quiet https://github.com/HeliXonProtein/OmegaFold.git /tmp/pip-req-build-51fbp8cw
  Resolved https://github.com/HeliXonProtein/OmegaFold.git to commit 313c873ad190b64506a497c926649e15fcd88fcd
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 707.3 kB/s eta 0:00:00
  Created wheel for OmegaFold: filename=OmegaFold-0.0.0-py3-none-any.whl size=55665 sha256=80d4f0c13eff0a557d03c2dd68cdc0e20924c8dcf1711f2c9d43c5aaf0c2fc86
  Stored in directory: /tmp/pip-ephem-wheel-cache-w98smny7/wheels/fe/57/01/3ce12996dd37debe3ee6a02e8748fffdd4df7b885c5bb8071d
Successfully built OmegaFold
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0+cu121
    Uninstalling torch-2.4.0+cu121:
      Successfully uninstalled torch-2.4.0+cu121
ERROR: pip's dependency resolver does not currently take into account al

In [3]:
#do a fake run of omegafold before mounting to drive to download the weights etc.
!omegafold /thisdoesnotexist.fa /content/output

INFO:root:Downloading weights from https://helixon.s3.amazonaws.com/release1.pt to /root/.cache/omegafold_ckpt/model.pt
100% 2.96G/2.96G [03:46<00:00, 14.0MB/s]
INFO:root:Constructing OmegaFold
INFO:root:Reading /thisdoesnotexist.fa
Traceback (most recent call last):
  File "/usr/local/bin/omegafold", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/omegafold/__main__.py", line 58, in main
    for i, (input_data, save_path) in enumerate(
  File "/usr/local/lib/python3.10/dist-packages/omegafold/pipeline.py", line 120, in fasta2inputs
    with open(fasta_path, 'r') as file:
FileNotFoundError: [Errno 2] No such file or directory: '/thisdoesnotexist.fa'


In [4]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
import time
from Bio import SeqIO

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/utilities/metadata_omegafold.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created omegafold metadata dataframe")

Existing generation metadata read in.


In [5]:
def count_records_fasta(fasta_file):
  records = list(SeqIO.parse(fasta_file, "fasta"))
  return len(records)

In [6]:
def digest_large_fasta(fasta_file,num_processed, max_num_records=100):
  !rm -rf tmp_files
  !mkdir tmp_files
  records = list(SeqIO.parse(fasta_file, "fasta"))
  records = records[num_processed:]
  digested = [records[i:i+max_num_records] for i in range(0, len(records), max_num_records)]
  tmp_fastas=[]
  for i, d in enumerate(digested):
    tmp_path = "/content/tmp_files/" + fasta_file.split('/')[-1].split('.')[0] + "_" + str(i) + ".fa"
    tmp_fastas.append(tmp_path)
    with open(tmp_path, "w") as f:
      SeqIO.write(d, f, "fasta")
  return tmp_fastas


In [7]:
for length in range(151,171):
  input_file_path = f"/content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len{length}.fa"
  output_dir_path = "/content/drive/MyDrive/Generative_Models/utilities/omegafold/" + input_file_path.split('/')[-1].split('.')[0]
  num_records = count_records_fasta(input_file_path)
  num_processed = all_metadata_df.loc[all_metadata_df['input_file_path'] == input_file_path, 'batch_size'].sum()
  if num_processed < num_records:
    print(input_file_path)
    print(f"{num_records} sequences.")
    print(f"{num_processed} already processed.")
    tmp_fastas = digest_large_fasta(input_file_path, num_processed)
    for tmp_fasta in tmp_fastas:
      meta_data_entry = pd.Series()
      meta_data_entry["model"] = "omegafold"
      meta_data_entry["task"] = "Structure Prediction"
      meta_data_entry["input_file_path"] = input_file_path
      meta_data_entry["output_dir_path"] = output_dir_path
      meta_data_entry["Timestamp"] = str(datetime.now())
      meta_data_entry['gpu'] = 'T4 GPU'
      meta_data_entry["batch_id"] = str(uuid.uuid4())
      meta_data_entry['batch_size'] = count_records_fasta(tmp_fasta)
      print(f"{meta_data_entry['batch_size']} sequences in this batch ({tmp_fasta})")
      print("-------------------------------------------------")
      start_time = time.time()
      !omegafold {tmp_fasta} {output_dir_path}
      end_time = time.time()
      total_job_time = end_time - start_time
      meta_data_entry['wall_time_batch'] = str(total_job_time) + " Seconds"
      meta_data_entry['wall_time_task'] = str(total_job_time/meta_data_entry['batch_size']) + " Seconds (inferred)"
      all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
      all_metadata_df.to_csv(meta_data_filepath, index=False)
  else:
    print(f"All entries in {input_file_path} already processed.")

All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len151.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len152.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len153.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len154.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len155.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len156.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len157.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len158.fa already processed.
All entries in /content/drive/MyDrive/Generative_Models/utilities/refold_inputs/all_len159.fa al

In [ ]:
input_file_path = f"/content/drive/MyDrive/Generative_Models/utilities/refold_inputs/bug_fixed.fa"
output_dir_path = "/content/drive/MyDrive/Generative_Models/utilities/omegafold/" + input_file_path.split('/')[-1].split('.')[0]
num_records = count_records_fasta(input_file_path)
num_processed = all_metadata_df.loc[all_metadata_df['input_file_path'] == input_file_path, 'batch_size'].sum()
if num_processed < num_records:
  print(input_file_path)
  print(f"{num_records} sequences.")
  print(f"{num_processed} already processed.")
  tmp_fastas = digest_large_fasta(input_file_path, num_processed)
  for tmp_fasta in tmp_fastas:
    meta_data_entry = pd.Series()
    meta_data_entry["model"] = "omegafold"
    meta_data_entry["task"] = "Structure Prediction"
    meta_data_entry["input_file_path"] = input_file_path
    meta_data_entry["output_dir_path"] = output_dir_path
    meta_data_entry["Timestamp"] = str(datetime.now())
    meta_data_entry['gpu'] = 'T4 GPU'
    meta_data_entry["batch_id"] = str(uuid.uuid4())
    meta_data_entry['batch_size'] = count_records_fasta(tmp_fasta)
    print(f"{meta_data_entry['batch_size']} sequences in this batch ({tmp_fasta})")
    print("-------------------------------------------------")
    start_time = time.time()
    !omegafold {tmp_fasta} {output_dir_path}
    end_time = time.time()
    total_job_time = end_time - start_time
    meta_data_entry['wall_time_batch'] = str(total_job_time) + " Seconds"
    meta_data_entry['wall_time_task'] = str(total_job_time/meta_data_entry['batch_size']) + " Seconds (inferred)"
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)
else:
  print(f"All entries in {input_file_path} already processed.")

In [ ]:
for length in range(14,151):
  input_file_path = f"/content/drive/MyDrive/Generative_Models/utilities/fold_inputs/all_len{length}.fa"
  print(input_file_path)
  print(count_records_fasta(input_file_path))

In [ ]:
!ls /content/drive/MyDrive/Generative_Models/Utilities/MPNN_backbone/len100/merged.fa

In [ ]:
meta_data_entry

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
# prompt: create a fasta file in the current directory from a list of protein sequences

def create_fasta_file(sequences, sequence_names, output_filename):
  with open(output_filename, 'w') as f:
    for i, sequence in enumerate(sequences):
      f.write('>sequence_{}\n'.format(i + 1))
      f.write(sequence + '\n')

# Example usage:
sequences = ["AGEEVRARLRRVGALALEAGAKADQSENSCTHFPGYLPNMLRDLRDAFSRTLPEIAAAVRDLLAALAAALGVPYPGGDPPVRDPEAALAALEEVMPQAENQDPDIKAHVISLGENLNTLRLRLRRCVAAGMRARGDAALADAALAAMEAARAALRTGDLAAKAAAARAAAAELAALVRRALAAV"]
sequence_names = ["IL10_3b027c98-d4af-451f-9aeb-e5b371429109"]
create_fasta_file(sequences, sequence_names, "foldy.fa")

In [ ]:
input_file_path = "/content/foldy.fa"
output_dir_path = "/content/"


!omegafold {input_file_path} {output_dir_path}

In [ ]:
# prompt: download file from current directory (google colab)

from google.colab import files
#files.download("protpardelle_"+sequence_names[0]+".pdb")
files.download("sequence_1.pdb")
